In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as ttsplit
import featuretools as ft
from catboost import CatBoostClassifier as Cat_clf
from catboost import CatBoostRegressor as Cat_reg
from collections import Counter
from sklearn.metrics import mean_absolute_error as mae

## обработаем данные

In [2]:
studs_info=pd.read_csv('onti-students-performance/studs_info.csv')
data=pd.read_csv('onti-students-performance/x_train.csv')

In [3]:
studs_info.head()

,STD_ID,number,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),,Образование,Дата выдачи,Что именно закончил
0,99136,13951.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2629.0,СР,2013-06-26 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
1,99112,13998.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,8240.0,СР,2012-06-23 00:00:00,"МОУ СОШ N 2, ПЕСТОВО"
2,99109,13999.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2314.0,СР,2013-06-25 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
3,99125,13953.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2345.0,СР,2013-06-23 00:00:00,"Гимназия ""ГАРМОНИЯ"""
4,100676,13950.0,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2650.0,СР,2012-06-29 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА


### studs

In [4]:
studs_info.columns

Index(['STD_ID', '   number', 'Пол', 'Статус', 'Дата выпуска',
       'Категория обучения', 'Форма обучения', 'Шифр',
       'направление (специальность)', '   ', 'Образование', 'Дата выдачи',
       'Что именно закончил'],
      dtype='object')

In [5]:
studs_info = studs_info.drop(['   number', '   '], axis = 1)  # видимо id

In [6]:
studs_info.head(20)

,STD_ID,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил
0,99136,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2013-06-26 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
1,99112,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2012-06-23 00:00:00,"МОУ СОШ N 2, ПЕСТОВО"
2,99109,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2013-06-25 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
3,99125,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2013-06-23 00:00:00,"Гимназия ""ГАРМОНИЯ"""
4,100676,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2012-06-29 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5,99099,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2013-06-21 00:00:00,"МОУ СОШ (полная), ХОЛМ"
6,103016,Ж,ВЫП,2019-06-30,ДОГ,Д,31.05.01,Лечебное дело,СР,2013-05-21 00:00:00,"СРЕДНЯЯ ШКОЛА N 1 , ЧУДОВО ул Титова"
7,100677,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,СР,2012-06-20 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
8,116989,М,ВЫП,2019-08-31,БП,Д,04.05.01,Фундаментальная и прикладная химия,СР,2014-06-20 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
9,120708,Ж,ВЫП,2019-06-30,ДОГ,Д,31.05.03,Стоматология,СР,2014-06-21 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА


In [7]:
date_cnt = Counter(studs_info['Дата выпуска'])
date_cnt.most_common()

[(nan, 5002), ('2019-08-31', 272), ('2019-06-30', 146), ('2020-01-31', 46)]

In [8]:
df = pd.DataFrame(studs_info)

In [9]:
df['Дата выпуска'] = studs_info['Дата выпуска'].fillna('lol')  # так не хорошо

По поводу **шифра** XX.YY.ZZ
XX - укрупненная группа утвержденная министерством образования
YY - обозначает на каком уровне учиться студент (02-Среднее профессиональное образование, 03-бакалавриат, 04-магистратура, 05-специалитет)
ZZ - уточненная группа по направлениям утвержденная университетом

In [10]:
def parse_cipher(s):
    s = str(s)
    if len(s) != 8:
        assert s == 'nan', s
        return pd.Series([0, 0, 0])
    return pd.Series([s[:2], s[3:5], s[6:]])

studs_info[['cipher1', 'cipher2', 'cipher3']] = studs_info['Шифр'].apply(parse_cipher)

In [11]:
cnt = Counter(df['Категория обучения'])
cnt.most_common()

[('ДОГ', 2779), ('БП', 2686), (nan, 1)]

In [12]:
cnt = Counter(df['Форма обучения'])
cnt.most_common()

[('Д', 3680), ('З', 1591), ('В', 194), (nan, 1)]

In [13]:
cnt = Counter(df['Образование'])
cnt.most_common()

[('СР', 4101),
 ('СП', 916),
 (nan, 191),
 ('ОО', 160),
 ('НП', 50),
 ('ОСН', 32),
 ('НС', 14),
 ('НВ', 2)]

In [14]:
def get_year(s):
    s = str(s)
    return s[:4]

studs_info['Дата выдачи'] = studs_info['Дата выдачи'].apply(get_year)

In [15]:
cnt = Counter(studs_info['cipher1'])
cnt.most_common()

[('31', 1322),
 ('44', 809),
 ('38', 542),
 ('40', 462),
 ('35', 276),
 ('08', 214),
 ('45', 191),
 ('11', 173),
 ('07', 159),
 ('15', 133),
 ('13', 126),
 ('42', 118),
 ('33', 104),
 ('09', 95),
 ('23', 86),
 ('37', 80),
 ('54', 78),
 ('21', 74),
 ('36', 73),
 ('05', 64),
 ('06', 50),
 ('04', 48),
 ('46', 45),
 ('29', 44),
 ('51', 36),
 ('01', 35),
 ('47', 19),
 ('34', 9),
 (0, 1)]

Переводим все в числа-категории

In [16]:
# sklearn.preprocessing.LabelEncoder() делает то же самое

df['Категория обучения'] = studs_info['Категория обучения'].astype('category').cat.codes
df['Пол'] = studs_info['Пол'].astype('category').cat.codes
df['Форма обучения'] = studs_info['Форма обучения'].astype('category').cat.codes
df['Образование'] = studs_info['Образование'].astype('category').cat.codes
df['Шифр'] = studs_info['Шифр'].astype('category').cat.codes
df['cipher1'] = studs_info['cipher1'].astype('category').cat.codes
df['cipher2'] = studs_info['cipher2'].astype('category').cat.codes
df['cipher3'] = studs_info['cipher3'].astype('category').cat.codes
df['направление (специальность)'] = studs_info['направление (специальность)'].astype('category').cat.codes
df['Дата выпуска'] = studs_info['Дата выпуска'].astype('category').cat.codes
df['Статус'] = studs_info['Статус'].astype('category').cat.codes
df['Что именно закончил'] = studs_info['Что именно закончил'].astype('category').cat.codes
df['Дата выдачи'] = studs_info['Дата выдачи'].astype('category').cat.codes

In [17]:
df = df.drop_duplicates('STD_ID')

In [18]:
df.head(5)

,STD_ID,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3
0,99136,0,0,1,0,1,24,17,6,27,39,14,3,1
1,99112,0,0,1,0,1,24,17,6,26,72,14,3,1
2,99109,0,0,1,0,1,24,17,6,27,39,14,3,1
3,99125,0,0,1,0,1,24,17,6,27,32,14,3,1
4,100676,1,0,1,0,1,24,17,6,26,39,14,3,1


### data

In [19]:
data.head()

,Unnamed: 0,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
0,51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг
1,72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг..."
2,96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,Экзамен,Химия
3,273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,Экзамен,Статистика
4,371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,Экзамен,Педиатрия


In [20]:
y=pd.read_csv('onti-students-performance/y_train.csv')
data['y'] = y['mark']

In [21]:
len(data)

105058

In [22]:
x_data = pd.DataFrame(data)

In [23]:
x_data.columns

Index(['Unnamed: 0', 'STD_ID', 'НАПРАВЛЕНИЕ', 'ГОД', 'КУРС', 'СЕМЕСТР',
       'АТТЕСТАЦИЯ', 'ДИСЦИПЛИНА', 'y'],
      dtype='object')

In [24]:
cnt = Counter(data['НАПРАВЛЕНИЕ'])
cnt.most_common()

[('31.05.01 Лечебное дело', 23378),
 ('31.05.03 Стоматология', 7996),
 ('44.03.05 Педагогическое образование (с двумя профилями подготовки)', 7996),
 ('40.03.01 Юриспруденция', 6036),
 ('38.03.01 Экономика', 5192),
 ('08.03.01 Строительство', 4213),
 ('44.03.01 Педагогическое образование', 3676),
 ('38.03.02 Менеджмент', 3344),
 ('07.03.01 Архитектура', 2893),
 ('45.03.02 Лингвистика', 2741),
 ('33.05.01 Фармация', 2721),
 ('07.03.03 Дизайн архитектурной среды', 2038),
 ('42.03.02 Журналистика', 2028),
 ('13.03.01 Теплоэнергетика и теплотехника', 1986),
 ('44.03.03 Специальное (дефектологическое) образование', 1928),
 ('38.03.03 Управление персоналом', 1628),
 ('11.03.03 Конструирование и технология электронных средств', 1564),
 ('09.03.01 Информатика и вычислительная техника', 1558),
 ('15.03.05 Конструкторско-технологическое обеспечение машиностроительных производств',
  1553),
 ('35.03.06 Агроинженерия', 1468),
 ('54.03.01 Дизайн', 1385),
 ('23.03.03 Эксплуатация транспортно-техноло

In [25]:
cnt = Counter(data['АТТЕСТАЦИЯ'])
cnt.most_common()

[('Экзамен', 60092),
 ('Дифференцированный зачет', 37791),
 ('Курсовая работа', 5522),
 ('Курсовой проект', 1631),
 ('Зачет', 22)]

In [26]:
cnt = Counter(data['ДИСЦИПЛИНА'])
len(cnt.most_common())

1400

In [27]:
x_data['НАПРАВЛЕНИЕ'] = data['НАПРАВЛЕНИЕ'].astype('category').cat.codes
x_data['ГОД'] = data['ГОД'].astype('category').cat.codes
x_data['АТТЕСТАЦИЯ'] = data['АТТЕСТАЦИЯ'].astype('category').cat.codes
x_data['ДИСЦИПЛИНА'] = data['ДИСЦИПЛИНА'].astype('category').cat.codes

In [331]:
agg_marks_students = x_data.groupby('STD_ID')['y'].agg([
    'sum','mean','std','min','max']).reset_index()
agg_marks_students.head()

,STD_ID,sum,mean,std,min,max
0,57258,32,3.555556,0.726483,3,5
1,58419,120,3.870968,0.718421,3,5
2,62101,111,3.827586,0.601722,3,5
3,65970,80,4.210526,0.713283,3,5
4,66520,81,3.681818,0.646335,3,5


In [332]:
agg_marks_dist = x_data.groupby('ДИСЦИПЛИНА')['y'].agg([
    'sum','mean','std','min','max']).reset_index()
agg_marks_napr = x_data.groupby('НАПРАВЛЕНИЕ')['y'].agg([
    'sum','mean','std','min','max']).reset_index()

In [333]:
dummy_data1 = {
        'id': ['1', '2', '5', '2', '7'],
        'Feature1': ['K', 'M', 'O', 'Q', 'S'],
        'Feature2': ['L', 'N', 'P', 'R', 'T']}
dummy_data2 = {
        'id': ['1', '2', '3', '4', '5'],
        'Feature12': ['Aqw', 'Cq', 'qE', 'qG', 'qI']}
df_1 = pd.DataFrame(dummy_data1, columns = ['id', 'Feature1', 'Feature2'])
df_2 = pd.DataFrame(dummy_data2, columns = ['id', 'Feature12'])
pd.merge(df_1, df_2, on='id')

,id,Feature1,Feature2,Feature12
0,1,K,L,Aqw
1,2,M,N,Cq
2,2,Q,R,Cq
3,5,O,P,qI


In [334]:
df = pd.merge(df, agg_marks_students, on='STD_ID')
df.head()

,STD_ID,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3,sum,mean,std,min,max
0,99136,0,0,1,0,1,24,17,6,27,39,14,3,1,209,4.644444,0.570309,3,5
1,99112,0,0,1,0,1,24,17,6,26,72,14,3,1,182,4.439024,0.634381,3,5
2,99109,0,0,1,0,1,24,17,6,27,39,14,3,1,214,4.863636,0.408680,3,5
3,99125,0,0,1,0,1,24,17,6,27,32,14,3,1,188,4.476190,0.505487,4,5
4,100676,1,0,1,0,1,24,17,6,26,39,14,3,1,169,4.225000,0.576795,3,5


In [335]:
x_data = pd.merge(x_data, agg_marks_dist, on='ДИСЦИПЛИНА')
x_data = pd.merge(x_data, agg_marks_napr, on='НАПРАВЛЕНИЕ')

In [28]:
x_data.head()

,Unnamed: 0,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,y
0,51081,175711,36,5,2.0,4.0,4,408,5
1,72921,100647,24,2,3.0,5.0,4,946,5
2,96438,199311,25,6,1.0,1.0,4,1323,5
3,273044,162002,36,4,2.0,3.0,4,1100,4
4,371309,108384,24,5,5.0,9.0,4,713,3


### feature tools

In [111]:
import featuretools as ft

In [112]:
df['index'] = df.index
x_data['index'] = x_data.index

In [ ]:
entity_set = ft.EntitySet(id = 'data')
entity_set = entity_set.entity_from_dataframe(
    entity_id='studs',
    dataframe=df,
    index='index',
    variable_types={
        'small_group': ft.variable_types.Categorical,
        'trans_date': ft.variable_types.Categorical,
    }
)
entity_set = entity_set.entity_from_dataframe(
    entity_id='data',
    dataframe=x_data,
    index='index',
    variable_types={
        'bins': ft.variable_types.Categorical
    }
)

### просто приписать данные ученика

In [35]:
train = pd.DataFrame(x_data)

In [36]:
train = pd.merge(train, df, on='STD_ID')

In [37]:
train = train.dropna()

In [270]:
train.corr()

,Unnamed: 0,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,y,Пол,...,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3,sum,mean,std,min,max
Unnamed: 0,1.000000,0.027982,0.042414,0.052004,0.036628,0.036298,0.024408,-0.012025,-0.072739,0.054292,...,-0.081621,0.003504,0.031040,-0.053873,0.015330,-0.040033,-0.114003,0.049730,-0.082421,-0.021171
STD_ID,0.027982,1.000000,0.078653,0.449289,-0.390630,-0.403675,-0.040034,-0.003550,-0.028109,0.030991,...,0.309513,0.038492,0.074975,-0.236076,0.018006,-0.649105,-0.043912,-0.016605,0.088796,-0.089828
НАПРАВЛЕНИЕ,0.042414,0.078653,1.000000,0.043056,-0.035327,-0.033637,0.027445,0.018875,0.083193,-0.251227,...,0.012500,0.046406,0.993105,-0.271523,0.181733,-0.189862,0.129963,-0.152670,0.167490,0.008959
ГОД,0.052004,0.449289,0.043056,1.000000,0.412243,0.375150,0.000830,0.092974,0.052800,0.043435,...,0.064517,0.082512,0.034117,-0.195002,0.042871,-0.477156,-0.042129,-0.024581,0.068204,-0.089318
КУРС,0.036628,-0.390630,-0.035327,0.412243,1.000000,0.979699,0.064901,0.103983,0.117374,-0.037911,...,-0.064918,-0.066230,-0.030021,0.167240,-0.032430,0.433912,0.051746,0.014106,-0.050944,0.091212
СЕМЕСТР,0.036298,-0.403675,-0.033637,0.375150,0.979699,1.000000,0.052108,0.101537,0.125656,-0.039887,...,-0.067757,-0.065882,-0.028285,0.163025,-0.031305,0.449434,0.053874,0.014304,-0.052426,0.096365
АТТЕСТАЦИЯ,0.024408,-0.040034,0.027445,0.000830,0.064901,0.052108,1.000000,-0.000853,-0.104850,-0.018373,...,0.006019,-0.021760,0.027195,0.078869,-0.011606,0.026739,0.006556,-0.009180,0.002250,0.007556
ДИСЦИПЛИНА,-0.012025,-0.003550,0.018875,0.092974,0.103983,0.101537,-0.000853,1.000000,-0.001653,0.043525,...,-0.021644,0.016514,0.020928,-0.008780,0.028909,-0.034959,-0.035421,0.007704,-0.009160,-0.014482
y,-0.072739,-0.028109,0.083193,0.052800,0.117374,0.125656,-0.104850,-0.001653,1.000000,-0.237472,...,0.034047,0.016810,0.076565,-0.002870,0.009397,0.219443,0.640127,-0.307103,0.437982,0.263276
Пол,0.054292,0.030991,-0.251227,0.043435,-0.037911,-0.039887,-0.018373,0.043525,-0.237472,1.000000,...,-0.075862,0.032093,-0.253047,-0.030322,-0.001325,-0.151371,-0.370977,0.152172,-0.219934,-0.158943


In [38]:
print(len(train))  # 100354 
train.head()

105058


,Unnamed: 0,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,y,Пол,...,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3
0,51081,175711,36,5,2.0,4.0,4,408,5,1,...,0,1,36,19,6,31,195,20,1,2
1,43778,175711,36,4,1.0,1.0,4,731,4,1,...,0,1,36,19,6,31,195,20,1,2
2,47411,175711,36,4,1.0,2.0,0,254,4,1,...,0,1,36,19,6,31,195,20,1,2
3,45690,175711,36,4,1.0,2.0,4,1154,5,1,...,0,1,36,19,6,31,195,20,1,2
4,43779,175711,36,4,1.0,2.0,0,417,4,1,...,0,1,36,19,6,31,195,20,1,2


## model

In [39]:
y_all = train.pop('y')

In [40]:
train = train.drop(['Unnamed: 0', 'STD_ID', 'Статус'], axis=1)

In [279]:
train = train.drop(['sum','mean','std','min','max'], axis=1)  # answer features

In [423]:
train = train.drop(['min_y', 'max_y', 'min_x', 'max_x'], 1)

In [60]:
x_train, x_val, y_train, y_val = ttsplit(train, y_all, test_size=0.2, random_state=42)

In [61]:
x_train.head()  # fix -1 

,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,Пол,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3
54279,8,4,2.0,4.0,3,1203,1,0,1,2,8,30,5,29,221,6,1,1
65221,18,5,2.0,3.0,0,230,1,3,0,1,18,14,6,31,39,10,1,5
54474,26,3,1.0,1.0,4,1280,0,3,0,1,26,35,6,30,39,15,3,1
98929,42,5,1.0,2.0,0,246,0,3,0,1,42,8,6,32,39,22,1,2
28291,25,5,4.0,7.0,4,583,0,3,0,1,25,29,6,29,182,14,3,3


In [62]:
x_train.columns

Index(['НАПРАВЛЕНИЕ', 'ГОД', 'КУРС', 'СЕМЕСТР', 'АТТЕСТАЦИЯ', 'ДИСЦИПЛИНА',
       'Пол', 'Дата выпуска', 'Категория обучения', 'Форма обучения', 'Шифр',
       'направление (специальность)', 'Образование', 'Дата выдачи',
       'Что именно закончил', 'cipher1', 'cipher2', 'cipher3'],
      dtype='object')

In [44]:
cat_feats = ['НАПРАВЛЕНИЕ', 'ГОД',
       'АТТЕСТАЦИЯ', 'ДИСЦИПЛИНА', 'Пол', 'Дата выпуска',
       'Категория обучения', 'Форма обучения', 'Шифр',
       'направление (специальность)', 'Образование', 'Дата выдачи', 'Что именно закончил', 'cipher1', 'cipher2', 'cipher3']

In [68]:
try:
    len(results)
except NameError:
    results = []

### CatBoost (regressor)

In [85]:
cat = Cat_reg(task_type='GPU', max_depth=10, n_estimators=3000, cat_features=cat_feats)

In [81]:
cat.fit(x_train, y_train)

0:	learn: 4.1269745	total: 96.5ms	remaining: 4m 49s
1:	learn: 4.0075336	total: 200ms	remaining: 4m 59s
2:	learn: 3.8915581	total: 300ms	remaining: 4m 59s
3:	learn: 3.7790825	total: 389ms	remaining: 4m 51s
4:	learn: 3.6714567	total: 472ms	remaining: 4m 42s
5:	learn: 3.5658440	total: 562ms	remaining: 4m 40s
6:	learn: 3.4637096	total: 661ms	remaining: 4m 42s
7:	learn: 3.3643942	total: 748ms	remaining: 4m 39s
8:	learn: 3.2679685	total: 846ms	remaining: 4m 41s
9:	learn: 3.1747223	total: 946ms	remaining: 4m 42s
10:	learn: 3.0851837	total: 1.03s	remaining: 4m 39s
11:	learn: 2.9979692	total: 1.11s	remaining: 4m 36s
12:	learn: 2.9133551	total: 1.2s	remaining: 4m 35s
13:	learn: 2.8314584	total: 1.29s	remaining: 4m 34s
14:	learn: 2.7523426	total: 1.38s	remaining: 4m 34s
15:	learn: 2.6754684	total: 1.47s	remaining: 4m 33s
16:	learn: 2.6011279	total: 1.53s	remaining: 4m 28s
17:	learn: 2.5289227	total: 1.63s	remaining: 4m 29s
18:	learn: 2.4592109	total: 1.71s	remaining: 4m 28s
19:	learn: 2.3921214	t

157:	learn: 0.5728065	total: 14s	remaining: 4m 11s
158:	learn: 0.5723642	total: 14.1s	remaining: 4m 11s
159:	learn: 0.5719689	total: 14.2s	remaining: 4m 11s
160:	learn: 0.5716344	total: 14.3s	remaining: 4m 11s
161:	learn: 0.5713870	total: 14.3s	remaining: 4m 11s
162:	learn: 0.5710122	total: 14.4s	remaining: 4m 11s
163:	learn: 0.5707176	total: 14.5s	remaining: 4m 10s
164:	learn: 0.5704056	total: 14.6s	remaining: 4m 10s
165:	learn: 0.5700704	total: 14.7s	remaining: 4m 10s
166:	learn: 0.5698198	total: 14.8s	remaining: 4m 10s
167:	learn: 0.5694115	total: 14.9s	remaining: 4m 10s
168:	learn: 0.5692223	total: 15s	remaining: 4m 10s
169:	learn: 0.5688673	total: 15.1s	remaining: 4m 10s
170:	learn: 0.5686060	total: 15.1s	remaining: 4m 10s
171:	learn: 0.5683639	total: 15.2s	remaining: 4m 10s
172:	learn: 0.5681615	total: 15.3s	remaining: 4m 10s
173:	learn: 0.5678996	total: 15.4s	remaining: 4m 10s
174:	learn: 0.5676331	total: 15.5s	remaining: 4m 9s
175:	learn: 0.5673395	total: 15.6s	remaining: 4m 10

315:	learn: 0.5389518	total: 27.6s	remaining: 3m 54s
316:	learn: 0.5388679	total: 27.7s	remaining: 3m 54s
317:	learn: 0.5386996	total: 27.8s	remaining: 3m 54s
318:	learn: 0.5384700	total: 27.9s	remaining: 3m 54s
319:	learn: 0.5383806	total: 28s	remaining: 3m 54s
320:	learn: 0.5383024	total: 28s	remaining: 3m 54s
321:	learn: 0.5382066	total: 28.1s	remaining: 3m 53s
322:	learn: 0.5381117	total: 28.2s	remaining: 3m 53s
323:	learn: 0.5380144	total: 28.3s	remaining: 3m 53s
324:	learn: 0.5377789	total: 28.4s	remaining: 3m 53s
325:	learn: 0.5376814	total: 28.5s	remaining: 3m 53s
326:	learn: 0.5374954	total: 28.6s	remaining: 3m 53s
327:	learn: 0.5374188	total: 28.7s	remaining: 3m 53s
328:	learn: 0.5373268	total: 28.8s	remaining: 3m 53s
329:	learn: 0.5371678	total: 28.9s	remaining: 3m 53s
330:	learn: 0.5368622	total: 29s	remaining: 3m 53s
331:	learn: 0.5366071	total: 29.1s	remaining: 3m 53s
332:	learn: 0.5364366	total: 29.1s	remaining: 3m 53s
333:	learn: 0.5363025	total: 29.2s	remaining: 3m 53s

471:	learn: 0.5150889	total: 41.1s	remaining: 3m 39s
472:	learn: 0.5149696	total: 41.1s	remaining: 3m 39s
473:	learn: 0.5148419	total: 41.2s	remaining: 3m 39s
474:	learn: 0.5147259	total: 41.3s	remaining: 3m 39s
475:	learn: 0.5145710	total: 41.4s	remaining: 3m 39s
476:	learn: 0.5144975	total: 41.5s	remaining: 3m 39s
477:	learn: 0.5143733	total: 41.6s	remaining: 3m 39s
478:	learn: 0.5142873	total: 41.7s	remaining: 3m 39s
479:	learn: 0.5141681	total: 41.7s	remaining: 3m 39s
480:	learn: 0.5139772	total: 41.8s	remaining: 3m 39s
481:	learn: 0.5138449	total: 41.9s	remaining: 3m 39s
482:	learn: 0.5136734	total: 42s	remaining: 3m 38s
483:	learn: 0.5134918	total: 42.1s	remaining: 3m 38s
484:	learn: 0.5132964	total: 42.2s	remaining: 3m 38s
485:	learn: 0.5131736	total: 42.3s	remaining: 3m 38s
486:	learn: 0.5131132	total: 42.3s	remaining: 3m 38s
487:	learn: 0.5130137	total: 42.4s	remaining: 3m 38s
488:	learn: 0.5127609	total: 42.5s	remaining: 3m 38s
489:	learn: 0.5126059	total: 42.6s	remaining: 3m

627:	learn: 0.4940713	total: 54.2s	remaining: 3m 24s
628:	learn: 0.4939496	total: 54.3s	remaining: 3m 24s
629:	learn: 0.4938023	total: 54.4s	remaining: 3m 24s
630:	learn: 0.4936836	total: 54.5s	remaining: 3m 24s
631:	learn: 0.4936392	total: 54.6s	remaining: 3m 24s
632:	learn: 0.4935180	total: 54.7s	remaining: 3m 24s
633:	learn: 0.4934144	total: 54.7s	remaining: 3m 24s
634:	learn: 0.4932468	total: 54.8s	remaining: 3m 24s
635:	learn: 0.4930848	total: 54.9s	remaining: 3m 24s
636:	learn: 0.4929741	total: 55s	remaining: 3m 23s
637:	learn: 0.4928643	total: 55.1s	remaining: 3m 23s
638:	learn: 0.4927445	total: 55.1s	remaining: 3m 23s
639:	learn: 0.4925674	total: 55.2s	remaining: 3m 23s
640:	learn: 0.4924660	total: 55.3s	remaining: 3m 23s
641:	learn: 0.4922498	total: 55.4s	remaining: 3m 23s
642:	learn: 0.4921065	total: 55.5s	remaining: 3m 23s
643:	learn: 0.4920072	total: 55.6s	remaining: 3m 23s
644:	learn: 0.4919413	total: 55.7s	remaining: 3m 23s
645:	learn: 0.4918238	total: 55.7s	remaining: 3m

...

2292:	learn: 0.3369450	total: 3m 18s	remaining: 1m 1s
2293:	learn: 0.3368595	total: 3m 18s	remaining: 1m 1s
2294:	learn: 0.3368393	total: 3m 18s	remaining: 1m
2295:	learn: 0.3367901	total: 3m 18s	remaining: 1m
2296:	learn: 0.3367387	total: 3m 18s	remaining: 1m
2297:	learn: 0.3366523	total: 3m 18s	remaining: 1m
2298:	learn: 0.3365906	total: 3m 18s	remaining: 1m
2299:	learn: 0.3365347	total: 3m 18s	remaining: 1m
2300:	learn: 0.3364578	total: 3m 18s	remaining: 1m
2301:	learn: 0.3363696	total: 3m 18s	remaining: 1m
2302:	learn: 0.3362875	total: 3m 19s	remaining: 1m
2303:	learn: 0.3362367	total: 3m 19s	remaining: 1m
2304:	learn: 0.3361855	total: 3m 19s	remaining: 1m
2305:	learn: 0.3360912	total: 3m 19s	remaining: 60s
2306:	learn: 0.3360156	total: 3m 19s	remaining: 59.9s
2307:	learn: 0.3359381	total: 3m 19s	remaining: 59.8s
2308:	learn: 0.3358610	total: 3m 19s	remaining: 59.7s
2309:	learn: 0.3358220	total: 3m 19s	remaining: 59.6s
2310:	learn: 0.3357504	total: 3m 19s	remaining: 59.5s
2311:	lea

2445:	learn: 0.3264926	total: 3m 31s	remaining: 47.9s
2446:	learn: 0.3263308	total: 3m 31s	remaining: 47.8s
2447:	learn: 0.3262780	total: 3m 31s	remaining: 47.7s
2448:	learn: 0.3261706	total: 3m 31s	remaining: 47.6s
2449:	learn: 0.3261179	total: 3m 31s	remaining: 47.6s
2450:	learn: 0.3260213	total: 3m 31s	remaining: 47.5s
2451:	learn: 0.3258920	total: 3m 31s	remaining: 47.4s
2452:	learn: 0.3258205	total: 3m 32s	remaining: 47.3s
2453:	learn: 0.3257393	total: 3m 32s	remaining: 47.2s
2454:	learn: 0.3256910	total: 3m 32s	remaining: 47.1s
2455:	learn: 0.3256254	total: 3m 32s	remaining: 47s
2456:	learn: 0.3255920	total: 3m 32s	remaining: 46.9s
2457:	learn: 0.3254662	total: 3m 32s	remaining: 46.9s
2458:	learn: 0.3254131	total: 3m 32s	remaining: 46.8s
2459:	learn: 0.3253320	total: 3m 32s	remaining: 46.7s
2460:	learn: 0.3252341	total: 3m 32s	remaining: 46.6s
2461:	learn: 0.3251595	total: 3m 32s	remaining: 46.5s
2462:	learn: 0.3250241	total: 3m 32s	remaining: 46.4s
2463:	learn: 0.3249773	total: 

2598:	learn: 0.3168044	total: 3m 44s	remaining: 34.7s
2599:	learn: 0.3167685	total: 3m 44s	remaining: 34.6s
2600:	learn: 0.3167301	total: 3m 44s	remaining: 34.5s
2601:	learn: 0.3166598	total: 3m 45s	remaining: 34.4s
2602:	learn: 0.3165534	total: 3m 45s	remaining: 34.3s
2603:	learn: 0.3165268	total: 3m 45s	remaining: 34.3s
2604:	learn: 0.3164341	total: 3m 45s	remaining: 34.2s
2605:	learn: 0.3163858	total: 3m 45s	remaining: 34.1s
2606:	learn: 0.3163351	total: 3m 45s	remaining: 34s
2607:	learn: 0.3162797	total: 3m 45s	remaining: 33.9s
2608:	learn: 0.3162041	total: 3m 45s	remaining: 33.8s
2609:	learn: 0.3161222	total: 3m 45s	remaining: 33.7s
2610:	learn: 0.3161091	total: 3m 45s	remaining: 33.7s
2611:	learn: 0.3160471	total: 3m 45s	remaining: 33.6s
2612:	learn: 0.3159987	total: 3m 46s	remaining: 33.5s
2613:	learn: 0.3158577	total: 3m 46s	remaining: 33.4s
2614:	learn: 0.3158212	total: 3m 46s	remaining: 33.3s
2615:	learn: 0.3156940	total: 3m 46s	remaining: 33.2s
2616:	learn: 0.3156586	total: 

2751:	learn: 0.3077644	total: 3m 58s	remaining: 21.5s
2752:	learn: 0.3077237	total: 3m 58s	remaining: 21.4s
2753:	learn: 0.3076952	total: 3m 58s	remaining: 21.3s
2754:	learn: 0.3076198	total: 3m 58s	remaining: 21.2s
2755:	learn: 0.3075695	total: 3m 58s	remaining: 21.1s
2756:	learn: 0.3075222	total: 3m 58s	remaining: 21s
2757:	learn: 0.3074312	total: 3m 58s	remaining: 20.9s
2758:	learn: 0.3073544	total: 3m 58s	remaining: 20.9s
2759:	learn: 0.3073189	total: 3m 58s	remaining: 20.8s
2760:	learn: 0.3072749	total: 3m 58s	remaining: 20.7s
2761:	learn: 0.3071512	total: 3m 59s	remaining: 20.6s
2762:	learn: 0.3071055	total: 3m 59s	remaining: 20.5s
2763:	learn: 0.3070482	total: 3m 59s	remaining: 20.4s
2764:	learn: 0.3069792	total: 3m 59s	remaining: 20.3s
2765:	learn: 0.3069292	total: 3m 59s	remaining: 20.3s
2766:	learn: 0.3068866	total: 3m 59s	remaining: 20.2s
2767:	learn: 0.3067956	total: 3m 59s	remaining: 20.1s
2768:	learn: 0.3066897	total: 3m 59s	remaining: 20s
2769:	learn: 0.3066489	total: 3m

2906:	learn: 0.2982282	total: 4m 11s	remaining: 8.05s
2907:	learn: 0.2981982	total: 4m 11s	remaining: 7.96s
2908:	learn: 0.2981720	total: 4m 11s	remaining: 7.88s
2909:	learn: 0.2981234	total: 4m 11s	remaining: 7.79s
2910:	learn: 0.2980898	total: 4m 11s	remaining: 7.7s
2911:	learn: 0.2980413	total: 4m 11s	remaining: 7.62s
2912:	learn: 0.2980142	total: 4m 12s	remaining: 7.53s
2913:	learn: 0.2979496	total: 4m 12s	remaining: 7.44s
2914:	learn: 0.2978985	total: 4m 12s	remaining: 7.36s
2915:	learn: 0.2978720	total: 4m 12s	remaining: 7.27s
2916:	learn: 0.2977748	total: 4m 12s	remaining: 7.18s
2917:	learn: 0.2977085	total: 4m 12s	remaining: 7.1s
2918:	learn: 0.2976570	total: 4m 12s	remaining: 7.01s
2919:	learn: 0.2975246	total: 4m 12s	remaining: 6.92s
2920:	learn: 0.2975087	total: 4m 12s	remaining: 6.84s
2921:	learn: 0.2974294	total: 4m 12s	remaining: 6.75s
2922:	learn: 0.2973770	total: 4m 12s	remaining: 6.66s
2923:	learn: 0.2972913	total: 4m 13s	remaining: 6.58s
2924:	learn: 0.2972226	total: 

In [82]:
results.append([mae(np.clip(cat.predict(x_val), 3, 5), y_val), mae(np.clip(cat.predict(x_train), 3, 5), y_train),
                'depth=10, est=3000, ' + str(len(cat_feats)) + ' cat_feats, no answer agg feats, decipher'])

In [83]:
#results[-1][-1] = 'depth=12, est=3000, ' + str(len(cat_feats)) + ' cat_feats, no answer agg feats, decipher'

In [84]:
results

[[0.4315645958414324,
  0.3715939131535825,
  'depth=10, est=5000, 16 cat_feats, no answer agg feats, decipher'],
 [0.43478993894827883,
  0.38045205845743235,
  'depth=10, est=3000, 16 cat_feats, no answer agg feats, decipher'],
 [0.43399696444250796,
  0.36658557782381,
  'depth=12, est=3000, 16 cat_feats, no answer agg feats, decipher']]

In [72]:
imp_feats = np.concatenate((cat.get_feature_importance().reshape(-1, 1), np.array(x_train.columns).reshape(-1, 1)), 1)
imp_feats = imp_feats[imp_feats[:,0].argsort()[::-1]]
imp_feats

array([[16.287245048252185, 'ДИСЦИПЛИНА'],
       [12.271747915821296, 'Что именно закончил'],
       [10.317340861356113, 'Дата выдачи'],
       [8.587923423634576, 'ГОД'],
       [5.780501461596198, 'Категория обучения'],
       [5.518566535872738, 'АТТЕСТАЦИЯ'],
       [5.484077623696611, 'cipher1'],
       [4.4076044427686885, 'Пол'],
       [4.06420599013918, 'cipher3'],
       [4.0170898163214765, 'Форма обучения'],
       [3.871485636301911, 'НАПРАВЛЕНИЕ'],
       [3.774735524776699, 'Дата выпуска'],
       [3.569150310097359, 'направление (специальность)'],
       [3.3829668898602443, 'СЕМЕСТР'],
       [2.987536449694362, 'Образование'],
       [2.3597499008498124, 'cipher2'],
       [2.2213873339725687, 'Шифр'],
       [1.0966848349878955, 'КУРС']], dtype=object)

## submission

In [86]:
x_test = pd.read_csv('onti-students-performance/x_test.csv')

In [87]:
len(x_test)

26265

In [88]:
x_test['НАПРАВЛЕНИЕ'] = x_test['НАПРАВЛЕНИЕ'].astype('category').cat.codes
x_test['ГОД'] = x_test['ГОД'].astype('category').cat.codes
x_test['АТТЕСТАЦИЯ'] = x_test['АТТЕСТАЦИЯ'].astype('category').cat.codes
x_test['ДИСЦИПЛИНА'] = x_test['ДИСЦИПЛИНА'].astype('category').cat.codes

In [89]:
#x_test = pd.merge(x_test, agg_marks_dist, on='ДИСЦИПЛИНА', how='left')
#x_test = pd.merge(x_test, agg_marks_napr, on='НАПРАВЛЕНИЕ', how='left')
x_test = pd.merge(x_test, df, on='STD_ID', validate='m:1', how='left')

In [90]:
x_test = x_test.drop(['Unnamed: 0', 'STD_ID', 'Статус'], axis=1)
# x_test = x_test.drop(['min_y', 'max_y', 'min_x', 'max_x'], 1)
x_test.head()

,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,Пол,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),Образование,Дата выдачи,Что именно закончил,cipher1,cipher2,cipher3
0,29,5,3.0,5.0,0,246,0,3,0,2,29,1,5,30,30,17,1,4
1,22,6,4.0,7.0,4,955,0,3,0,1,22,41,6,30,39,12,1,3
2,40,5,1.0,2.0,4,255,1,3,1,1,40,43,6,32,39,21,1,1
3,35,5,2.0,3.0,0,218,1,3,1,1,35,40,6,31,195,20,1,1
4,47,5,3.0,6.0,4,1201,0,3,0,1,47,22,6,30,59,23,1,5


In [91]:
assert len(train.columns) == len( x_test.columns)

In [92]:
cat.fit(train, y_all)

0:	learn: 4.1273753	total: 62.2ms	remaining: 3m 6s
1:	learn: 4.0077339	total: 123ms	remaining: 3m 3s
2:	learn: 3.8913749	total: 192ms	remaining: 3m 11s
3:	learn: 3.7784937	total: 265ms	remaining: 3m 18s
4:	learn: 3.6690280	total: 326ms	remaining: 3m 15s
5:	learn: 3.5636280	total: 386ms	remaining: 3m 12s
6:	learn: 3.4612135	total: 447ms	remaining: 3m 11s
7:	learn: 3.3620066	total: 509ms	remaining: 3m 10s
8:	learn: 3.2654485	total: 570ms	remaining: 3m 9s
9:	learn: 3.1724225	total: 628ms	remaining: 3m 7s
10:	learn: 3.0819806	total: 696ms	remaining: 3m 9s
11:	learn: 2.9944785	total: 755ms	remaining: 3m 8s
12:	learn: 2.9093537	total: 816ms	remaining: 3m 7s
13:	learn: 2.8267912	total: 874ms	remaining: 3m 6s
14:	learn: 2.7469030	total: 937ms	remaining: 3m 6s
15:	learn: 2.6696471	total: 998ms	remaining: 3m 6s
16:	learn: 2.5948167	total: 1.06s	remaining: 3m 6s
17:	learn: 2.5223047	total: 1.12s	remaining: 3m 6s
18:	learn: 2.4521694	total: 1.18s	remaining: 3m 5s
19:	learn: 2.3843034	total: 1.25s	

160:	learn: 0.5827003	total: 9.44s	remaining: 2m 46s
161:	learn: 0.5824165	total: 9.5s	remaining: 2m 46s
162:	learn: 0.5821740	total: 9.57s	remaining: 2m 46s
163:	learn: 0.5818731	total: 9.63s	remaining: 2m 46s
164:	learn: 0.5816291	total: 9.69s	remaining: 2m 46s
165:	learn: 0.5814186	total: 9.74s	remaining: 2m 46s
166:	learn: 0.5811724	total: 9.8s	remaining: 2m 46s
167:	learn: 0.5809624	total: 9.85s	remaining: 2m 46s
168:	learn: 0.5807130	total: 9.91s	remaining: 2m 46s
169:	learn: 0.5805259	total: 9.97s	remaining: 2m 45s
170:	learn: 0.5803599	total: 10s	remaining: 2m 45s
171:	learn: 0.5802072	total: 10.1s	remaining: 2m 45s
172:	learn: 0.5800142	total: 10.1s	remaining: 2m 45s
173:	learn: 0.5798641	total: 10.2s	remaining: 2m 45s
174:	learn: 0.5797390	total: 10.2s	remaining: 2m 45s
175:	learn: 0.5795542	total: 10.3s	remaining: 2m 45s
176:	learn: 0.5793853	total: 10.3s	remaining: 2m 44s
177:	learn: 0.5791980	total: 10.4s	remaining: 2m 44s
178:	learn: 0.5789640	total: 10.5s	remaining: 2m 4

316:	learn: 0.5631144	total: 17.8s	remaining: 2m 30s
317:	learn: 0.5630396	total: 17.9s	remaining: 2m 30s
318:	learn: 0.5628984	total: 17.9s	remaining: 2m 30s
319:	learn: 0.5628288	total: 18s	remaining: 2m 30s
320:	learn: 0.5627441	total: 18s	remaining: 2m 30s
321:	learn: 0.5626790	total: 18.1s	remaining: 2m 30s
322:	learn: 0.5626299	total: 18.2s	remaining: 2m 30s
323:	learn: 0.5625408	total: 18.2s	remaining: 2m 30s
324:	learn: 0.5624029	total: 18.3s	remaining: 2m 30s
325:	learn: 0.5623135	total: 18.3s	remaining: 2m 30s
326:	learn: 0.5622357	total: 18.4s	remaining: 2m 30s
327:	learn: 0.5621419	total: 18.4s	remaining: 2m 30s
328:	learn: 0.5621110	total: 18.5s	remaining: 2m 30s
329:	learn: 0.5620702	total: 18.5s	remaining: 2m 29s
330:	learn: 0.5620162	total: 18.6s	remaining: 2m 29s
331:	learn: 0.5619434	total: 18.7s	remaining: 2m 29s
332:	learn: 0.5618770	total: 18.7s	remaining: 2m 29s
333:	learn: 0.5618542	total: 18.8s	remaining: 2m 29s
334:	learn: 0.5617861	total: 18.8s	remaining: 2m 2

472:	learn: 0.5524012	total: 26.2s	remaining: 2m 20s
473:	learn: 0.5523493	total: 26.3s	remaining: 2m 20s
474:	learn: 0.5522928	total: 26.3s	remaining: 2m 20s
475:	learn: 0.5522215	total: 26.4s	remaining: 2m 19s
476:	learn: 0.5521139	total: 26.5s	remaining: 2m 19s
477:	learn: 0.5520456	total: 26.5s	remaining: 2m 19s
478:	learn: 0.5520078	total: 26.6s	remaining: 2m 19s
479:	learn: 0.5519638	total: 26.6s	remaining: 2m 19s
480:	learn: 0.5518931	total: 26.7s	remaining: 2m 19s
481:	learn: 0.5518455	total: 26.7s	remaining: 2m 19s
482:	learn: 0.5518009	total: 26.8s	remaining: 2m 19s
483:	learn: 0.5517071	total: 26.8s	remaining: 2m 19s
484:	learn: 0.5516661	total: 26.9s	remaining: 2m 19s
485:	learn: 0.5516192	total: 26.9s	remaining: 2m 19s
486:	learn: 0.5515719	total: 27s	remaining: 2m 19s
487:	learn: 0.5514990	total: 27s	remaining: 2m 19s
488:	learn: 0.5514325	total: 27.1s	remaining: 2m 19s
489:	learn: 0.5513742	total: 27.1s	remaining: 2m 18s
490:	learn: 0.5513245	total: 27.2s	remaining: 2m 1

628:	learn: 0.5435285	total: 34.5s	remaining: 2m 9s
629:	learn: 0.5434950	total: 34.5s	remaining: 2m 9s
630:	learn: 0.5434087	total: 34.6s	remaining: 2m 9s
631:	learn: 0.5433824	total: 34.7s	remaining: 2m 9s
632:	learn: 0.5433098	total: 34.7s	remaining: 2m 9s
633:	learn: 0.5432970	total: 34.8s	remaining: 2m 9s
634:	learn: 0.5432516	total: 34.8s	remaining: 2m 9s
635:	learn: 0.5432075	total: 34.9s	remaining: 2m 9s
636:	learn: 0.5431539	total: 35s	remaining: 2m 9s
637:	learn: 0.5431298	total: 35s	remaining: 2m 9s
638:	learn: 0.5430344	total: 35.1s	remaining: 2m 9s
639:	learn: 0.5429572	total: 35.1s	remaining: 2m 9s
640:	learn: 0.5429146	total: 35.2s	remaining: 2m 9s
641:	learn: 0.5428611	total: 35.2s	remaining: 2m 9s
642:	learn: 0.5427847	total: 35.3s	remaining: 2m 9s
643:	learn: 0.5427248	total: 35.3s	remaining: 2m 9s
644:	learn: 0.5426603	total: 35.4s	remaining: 2m 9s
645:	learn: 0.5426326	total: 35.4s	remaining: 2m 9s
646:	learn: 0.5425738	total: 35.5s	remaining: 2m 9s
647:	learn: 0.54

...

2314:	learn: 0.4686962	total: 2m 6s	remaining: 37.4s
2315:	learn: 0.4686626	total: 2m 6s	remaining: 37.4s
2316:	learn: 0.4686239	total: 2m 6s	remaining: 37.3s
2317:	learn: 0.4685939	total: 2m 6s	remaining: 37.3s
2318:	learn: 0.4685705	total: 2m 6s	remaining: 37.2s
2319:	learn: 0.4685375	total: 2m 6s	remaining: 37.1s
2320:	learn: 0.4685241	total: 2m 6s	remaining: 37.1s
2321:	learn: 0.4684931	total: 2m 6s	remaining: 37s
2322:	learn: 0.4684750	total: 2m 6s	remaining: 37s
2323:	learn: 0.4684556	total: 2m 6s	remaining: 36.9s
2324:	learn: 0.4684192	total: 2m 6s	remaining: 36.9s
2325:	learn: 0.4683843	total: 2m 7s	remaining: 36.8s
2326:	learn: 0.4683620	total: 2m 7s	remaining: 36.8s
2327:	learn: 0.4683369	total: 2m 7s	remaining: 36.7s
2328:	learn: 0.4682952	total: 2m 7s	remaining: 36.6s
2329:	learn: 0.4682596	total: 2m 7s	remaining: 36.6s
2330:	learn: 0.4682409	total: 2m 7s	remaining: 36.5s
2331:	learn: 0.4681948	total: 2m 7s	remaining: 36.5s
2332:	learn: 0.4681459	total: 2m 7s	remaining: 36.

2468:	learn: 0.4635587	total: 2m 14s	remaining: 29s
2469:	learn: 0.4635228	total: 2m 14s	remaining: 28.9s
2470:	learn: 0.4635104	total: 2m 14s	remaining: 28.9s
2471:	learn: 0.4634744	total: 2m 15s	remaining: 28.8s
2472:	learn: 0.4634432	total: 2m 15s	remaining: 28.8s
2473:	learn: 0.4634171	total: 2m 15s	remaining: 28.7s
2474:	learn: 0.4633957	total: 2m 15s	remaining: 28.7s
2475:	learn: 0.4633755	total: 2m 15s	remaining: 28.6s
2476:	learn: 0.4633582	total: 2m 15s	remaining: 28.6s
2477:	learn: 0.4632949	total: 2m 15s	remaining: 28.5s
2478:	learn: 0.4632372	total: 2m 15s	remaining: 28.5s
2479:	learn: 0.4631918	total: 2m 15s	remaining: 28.4s
2480:	learn: 0.4631722	total: 2m 15s	remaining: 28.3s
2481:	learn: 0.4631537	total: 2m 15s	remaining: 28.3s
2482:	learn: 0.4631031	total: 2m 15s	remaining: 28.2s
2483:	learn: 0.4630621	total: 2m 15s	remaining: 28.2s
2484:	learn: 0.4630189	total: 2m 15s	remaining: 28.1s
2485:	learn: 0.4629678	total: 2m 15s	remaining: 28.1s
2486:	learn: 0.4629525	total: 

2621:	learn: 0.4579902	total: 2m 23s	remaining: 20.6s
2622:	learn: 0.4579361	total: 2m 23s	remaining: 20.6s
2623:	learn: 0.4578733	total: 2m 23s	remaining: 20.5s
2624:	learn: 0.4578552	total: 2m 23s	remaining: 20.5s
2625:	learn: 0.4578344	total: 2m 23s	remaining: 20.4s
2626:	learn: 0.4577934	total: 2m 23s	remaining: 20.4s
2627:	learn: 0.4577453	total: 2m 23s	remaining: 20.3s
2628:	learn: 0.4577149	total: 2m 23s	remaining: 20.3s
2629:	learn: 0.4576758	total: 2m 23s	remaining: 20.2s
2630:	learn: 0.4576223	total: 2m 23s	remaining: 20.1s
2631:	learn: 0.4575929	total: 2m 23s	remaining: 20.1s
2632:	learn: 0.4575621	total: 2m 23s	remaining: 20s
2633:	learn: 0.4575382	total: 2m 23s	remaining: 20s
2634:	learn: 0.4575053	total: 2m 23s	remaining: 19.9s
2635:	learn: 0.4574710	total: 2m 23s	remaining: 19.9s
2636:	learn: 0.4574521	total: 2m 23s	remaining: 19.8s
2637:	learn: 0.4574162	total: 2m 23s	remaining: 19.8s
2638:	learn: 0.4573777	total: 2m 24s	remaining: 19.7s
2639:	learn: 0.4573231	total: 2m

2774:	learn: 0.4521742	total: 2m 31s	remaining: 12.3s
2775:	learn: 0.4521512	total: 2m 31s	remaining: 12.2s
2776:	learn: 0.4521173	total: 2m 31s	remaining: 12.2s
2777:	learn: 0.4520710	total: 2m 31s	remaining: 12.1s
2778:	learn: 0.4520298	total: 2m 31s	remaining: 12.1s
2779:	learn: 0.4519981	total: 2m 31s	remaining: 12s
2780:	learn: 0.4519877	total: 2m 31s	remaining: 12s
2781:	learn: 0.4519517	total: 2m 31s	remaining: 11.9s
2782:	learn: 0.4519381	total: 2m 31s	remaining: 11.8s
2783:	learn: 0.4518815	total: 2m 32s	remaining: 11.8s
2784:	learn: 0.4518350	total: 2m 32s	remaining: 11.7s
2785:	learn: 0.4517678	total: 2m 32s	remaining: 11.7s
2786:	learn: 0.4517476	total: 2m 32s	remaining: 11.6s
2787:	learn: 0.4517121	total: 2m 32s	remaining: 11.6s
2788:	learn: 0.4516769	total: 2m 32s	remaining: 11.5s
2789:	learn: 0.4515999	total: 2m 32s	remaining: 11.5s
2790:	learn: 0.4515545	total: 2m 32s	remaining: 11.4s
2791:	learn: 0.4515247	total: 2m 32s	remaining: 11.4s
2792:	learn: 0.4514971	total: 2m

2927:	learn: 0.4470138	total: 2m 40s	remaining: 3.94s
2928:	learn: 0.4469713	total: 2m 40s	remaining: 3.88s
2929:	learn: 0.4469260	total: 2m 40s	remaining: 3.83s
2930:	learn: 0.4468928	total: 2m 40s	remaining: 3.77s
2931:	learn: 0.4468547	total: 2m 40s	remaining: 3.72s
2932:	learn: 0.4468220	total: 2m 40s	remaining: 3.66s
2933:	learn: 0.4467434	total: 2m 40s	remaining: 3.61s
2934:	learn: 0.4466807	total: 2m 40s	remaining: 3.55s
2935:	learn: 0.4466238	total: 2m 40s	remaining: 3.5s
2936:	learn: 0.4466031	total: 2m 40s	remaining: 3.44s
2937:	learn: 0.4465906	total: 2m 40s	remaining: 3.39s
2938:	learn: 0.4465419	total: 2m 40s	remaining: 3.33s
2939:	learn: 0.4464959	total: 2m 40s	remaining: 3.28s
2940:	learn: 0.4464865	total: 2m 40s	remaining: 3.23s
2941:	learn: 0.4464299	total: 2m 40s	remaining: 3.17s
2942:	learn: 0.4464017	total: 2m 40s	remaining: 3.12s
2943:	learn: 0.4463783	total: 2m 40s	remaining: 3.06s
2944:	learn: 0.4463179	total: 2m 40s	remaining: 3.01s
2945:	learn: 0.4463004	total:

In [93]:
ans = pd.read_csv("onti-students-performance/sample_submission.csv",index_col=0)

In [94]:
len(ans)

26265

In [95]:
len(x_test)

26265

In [96]:
x_test = x_test.fillna(0)

In [97]:
ans['mark'] = np.clip(cat.predict(x_test), 3, 5)

In [98]:
ans.to_csv("submissions/catBoost_0.36653_10_5000.csv")

In [99]:
mae(np.clip(cat.predict(train), 3, 5), y_all)

0.3786464582155274